In [ ]:
from segment_anything import sam_model_registry, SamPredictor
import numpy as np
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import os

model = YOLO(r"C:\Users\laksh\Desktop\dataset\a\runs\detect\train14\weights\best.pt")

test_img_path = r"C:\Users\laksh\Desktop\dataset\a\dataset\images\train\Lucknow_42880_48640.jpg"
img = cv2.imread(test_img_path)
lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
l, a, b = cv2.split(lab)
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
cl = clahe.apply(l)
limg = cv2.merge((cl, a, b))
img_rgb = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)

results = model(img_rgb)
boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)

sam = sam_model_registry["vit_h"](checkpoint="sam_vit_h_4b8939.pth")
predictor = SamPredictor(sam)
predictor.set_image(img_rgb)

masks = []
for box in boxes:
    x0, y0, x1, y1 = box
    input_box = np.array([x0, y0, x1, y1])
    mask, scores, _ = predictor.predict(box=input_box, multimask_output=True)
    best_idx = np.argmax(scores)
    masks.append(mask[best_idx].squeeze())

overlay = img_rgb.copy()
for m in masks:
    overlay[m] = [0, 255, 0]  # Green mask overlay


plt.imshow(overlay)
plt.axis('off')
plt.title("YOLOv8 + SAM Segmentation")
plt.show()